In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 675kB 6.4MB/s 
     |████████████████████████████████| 1.1MB 13.6MB/s 
     |████████████████████████████████| 3.8MB 17.2MB/s 
     |████████████████████████████████| 890kB 39.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=5a939c541fb335a1d65390e39568e4ba02f170b700e4c22cafbad4b0310c5c34
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f3c7e11fb338014870ace1594b5135d44f7e06ac7073b01e963aa5e2f8f6b06e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
root = '/content/drive/My Drive/nlp-lab'

######Change here depending on the dataset and model##########################
dataset_name = 'amazon-english'
#dataset_name = 'amazon-german2'
#dataset_name = 'organic'

#Huggingface models -> https://huggingface.co/transformers/multilingual.html
#model_name='xlm-mlm-ende-1024' #XLM (Masked language modeling, English-German)
model_name='xlm-roberta-base'
##############################################################################

processed_data_location = root + '/data/processed-data/' + dataset_name + '/processed_data_without_embeddings_no_outliers.json'

exported_file_name = dataset_name + '_with_embeddings_' + model_name
exported_file_location = root + '/data/data-with-embeddings/' +  model_name + '/' + exported_file_name





In [ ]:
import pandas as pd
from google.colab import drive
from sys import path

In [ ]:
drive.mount('/content/drive', force_remount=True)
path.append(root)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data_df = pd.read_json(processed_data_location)

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models

In [ ]:
#sentence-transformer options
#model = SentenceTransformer('bert-base-nli-mean-tokens')

#Huggingface options
#word_embedding_model = models.CamemBERT('camembert-base')
word_embedding_model = models.Transformer(model_name)
dim = word_embedding_model.get_word_embedding_dimension()
pooling_model = models.Pooling(dim, pooling_mode_mean_tokens=True, pooling_mode_cls_token=False, pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
print('loaded model: ' + model_name)


loaded model: xlm-roberta-base


In [ ]:
sentences_encoded = model.encode(data_df['sentence_text'].tolist())

In [ ]:
print(len(sentences_encoded))
print(len(sentences_encoded[0]))

129882
768


In [ ]:
data_with_embeddings = pd.concat([data_df.reset_index(drop=True),
                                  pd.DataFrame(sentences_encoded)], axis=1)

In [ ]:
#Split the dataframe in subchunks (for large dataframes)
def split_data_frame(df, chunk_size = 10000): 
    df_list = list()
    number_of_chunks = len(df) // chunk_size + 1
    for i in range(number_of_chunks):
        df_list.append(df[i*chunk_size:(i+1)*chunk_size])
    return df_list

df_list = split_data_frame(data_with_embeddings,20000)

In [ ]:
#For small dataframes:
#data_with_embeddings.to_json(exported_file_location + '.json')

#For large dataframes:
for i, df in enumerate(df_list):
  chunk_location = exported_file_location + str(i) + '.json'
  df.to_json(chunk_location)